In [1]:
from glob import glob
import librosa
import soundfile as sf
import numpy as np
import pandas as pd
import sys
import time
import datetime
from tqdm import tqdm
import yaml
import random
import os
import gc
import cv2

from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, AdamW, lr_scheduler
from torch.distributions import Uniform
from torch.utils.data import DataLoader, Dataset

from torchlibrosa.stft import Spectrogram, LogmelFilterBank
from torchlibrosa.augmentation import SpecAugmentation

from efficientnet_pytorch import EfficientNet
from torchvision.models import resnet34, resnet50

import sys
sys.path.append('..')
from libs import transform as tr
from libs import spectrogram as spec
from libs import criterion as cr
from libs import wormup as wu


import warnings
warnings.filterwarnings("ignore")

/home/yuigahama/anaconda3/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


In [2]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = False  # type: ignore

set_seed(42)

In [3]:
with open('../settings/efficient_b3_nosed.yaml') as f:
    settings = yaml.load(f, Loader=yaml.FullLoader)

In [4]:
def normalize_melspec(X: np.ndarray):
    eps = 1e-6
    mean = X.mean()
    X = X - mean
    std = X.std()
    Xstd = X / (std + eps)
    norm_min, norm_max = Xstd.min(), Xstd.max()
    if (norm_max - norm_min) > eps:
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V

def save(fold, model, optim, criterion, file_path="../../model/"):
    if not settings['test_name'] in os.listdir(file_path):
        os.mkdir(file_path+settings['test_name'])
    
    
    output_path = file_path + settings['test_name'] + '/' + f"{settings['test_name']}_{fold}.model"
    
    torch.save(
        {
            'epoch': epoch,
            'model_state_dict': model.cpu().state_dict(),
            'optimizer_state_dict': optim.state_dict(),
            'criterion': criterion
        },
        output_path)
    
    model.to(device)
    
    return output_path

In [5]:
class RfcxDataSet(Dataset):
    def __init__(self,
                 tp:pd.DataFrame,
                 train: bool,
                 data_path:str,
                 transform_params:dict,
                 pre_calc=True,
                 n_mels=128
    ):
        self.tp = tp
        self.path = data_path
        self.img_size = 256
        self.train = train
        self.n_mels = n_mels
        self.pre_calc = pre_calc
        
        self.transform = tr.Compose([
            #tr.OneOf([
            #    tr.GaussianNoiseSNR(min_snr=transform_params['GaussianNoiseSNR']),
            #    tr.PinkNoiseSNR(min_snr=transform_params['PinkNoiseSNR'])
            #]),
            tr.PitchShift(max_steps=transform_params['max_steps'], sr=SR),
            #tr.TimeStretch(),
            #tr.TimeShift(sr=sr),
            tr.VolumeControl(mode="sine")
        ])
        
        
    def __len__(self):
        return len(self.tp)
    
    def load(self, record_path):
        y, orig_sr = sf.read(record_path)
        
        if orig_sr != SR:
            y = librosa.resample(y, orig_sr=orig_sr, target_sr=SR, res_type="kaiser_best")
        return y
    
    def get_random_duration(self, duration=10):
        start_sec = random.randint(0, 60-duration)
        end_sec = start_sec + 10
            
        return start_sec, end_sec
    
    def get_duration(self, t_min, t_max, duration=10):
        annotated_duration = t_max - t_min
        
        if annotated_duration > duration:
            limit_sec = t_max - duration
            start_sec = random.randint(t_min, limit_sec)
            end_sec = start_sec + duration

        else:
            res_time = duration - annotated_duration
            front_limit = res_time if res_time < t_min else t_min
            
            front_time = random.randint(0, front_limit)
            
            back_limit = 60 - t_max
            
            tmp_time = res_time - front_time
            back_time = tmp_time if tmp_time < back_limit else back_limit
            
            if not tmp_time < back_limit:
                front_time += tmp_time - back_limit
            
            start_sec = t_min - front_time
            end_sec = t_max + back_time
            
        return start_sec, end_sec
    
    def create_mel(self, y):
        y = self.transform(y)
        
        melspec = librosa.feature.melspectrogram(
            y,
            sr=SR,
            fmin=0,
            fmax=15000,
            n_mels=128
        )

        pcen = librosa.pcen(melspec, sr=SR, **self.pcen_parameters)
        clean_mel = librosa.power_to_db(melspec ** 1.5)
        melspec = librosa.power_to_db(melspec)

        norm_melspec = normalize_melspec(melspec)
        norm_pcen = normalize_melspec(pcen)
        norm_clean_mel = normalize_melspec(clean_mel)

        image = np.stack([norm_melspec, norm_pcen, norm_clean_mel], axis=-1)

        return image
    
    def __getitem__(self, idx: int):
        sample = self.tp.iloc[idx, :]
        recording_id = sample['recording_id']
        t_min = int(round(sample['t_min']))
        t_max = int(round(sample['t_max']))
        
        start_sec, end_sec = self.get_duration(t_min, t_max, settings['duration'])
            
        record_path = self.path + recording_id + '.flac'
        y = self.load(record_path)
        y =  y[start_sec*SR:end_sec*SR]
        
        if self.train:
            y = self.transform(y)
        
        species_id = sample['species_id']
        target = torch.zeros([24], dtype=torch.float32)
        target[species_id] = 1
        
        return y, target

In [6]:
def init_layer(layer):
    nn.init.xavier_uniform_(layer.weight)

    if hasattr(layer, "bias"):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)


def init_bn(bn):
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.0)


def init_weights(model):
    classname = model.__class__.__name__
    if classname.find("Conv2d") != -1:
        nn.init.xavier_uniform_(model.weight, gain=np.sqrt(2))
        model.bias.data.fill_(0)
    elif classname.find("BatchNorm") != -1:
        model.weight.data.normal_(1.0, 0.02)
        model.bias.data.fill_(0)
    elif classname.find("GRU") != -1:
        for weight in model.parameters():
            if len(weight.size()) > 1:
                nn.init.orghogonal_(weight.data)
    elif classname.find("Linear") != -1:
        model.weight.data.normal_(0, 0.01)
        model.bias.data.zero_()


def do_mixup(x: torch.Tensor, mixup_lambda: torch.Tensor):
    """Mixup x of even indexes (0, 2, 4, ...) with x of odd indexes
    (1, 3, 5, ...).
    Args:
      x: (batch_size * 2, ...)
      mixup_lambda: (batch_size * 2,)
    Returns:
      out: (batch_size, ...)
    """
    out = (x[0::2].transpose(0, -1) * mixup_lambda[0::2] +
           x[1::2].transpose(0, -1) * mixup_lambda[1::2]).transpose(0, -1)
    return out


class Mixup(object):
    def __init__(self, mixup_alpha, random_seed=1234):
        """Mixup coefficient generator.
        """
        self.mixup_alpha = mixup_alpha
        self.random_state = np.random.RandomState(random_seed)

    def get_lambda(self, batch_size):
        """Get mixup random coefficients.
        Args:
          batch_size: int
        Returns:
          mixup_lambdas: (batch_size,)
        """
        mixup_lambdas = []
        for n in range(0, batch_size, 2):
            lam = self.random_state.beta(self.mixup_alpha, self.mixup_alpha, 1)[0]
            mixup_lambdas.append(lam)
            mixup_lambdas.append(1. - lam)

        return torch.from_numpy(np.array(mixup_lambdas, dtype=np.float32))


def interpolate(x: torch.Tensor, ratio: int):
    """Interpolate data in time domain. This is used to compensate the
    resolution reduction in downsampling of a CNN.
    Args:
      x: (batch_size, time_steps, classes_num)
      ratio: int, ratio to interpolate
    Returns:
      upsampled: (batch_size, time_steps * ratio, classes_num)
    """
    (batch_size, time_steps, classes_num) = x.shape
    upsampled = x[:, :, None, :].repeat(1, 1, ratio, 1)
    upsampled = upsampled.reshape(batch_size, time_steps * ratio, classes_num)
    return upsampled


def pad_framewise_output(framewise_output: torch.Tensor, frames_num: int):
    """Pad framewise_output to the same length as input frames. The pad value
    is the same as the value of the last frame.
    Args:
      framewise_output: (batch_size, frames_num, classes_num)
      frames_num: int, number of frames to pad
    Outputs:
      output: (batch_size, frames_num, classes_num)
    """
    pad = framewise_output[:, -1:, :].repeat(
        1, frames_num - framewise_output.shape[1], 1)
    """tensor for padding"""

    output = torch.cat((framewise_output, pad), dim=1)
    """(batch_size, frames_num, classes_num)"""

    return output

In [7]:
class AttBlock(nn.Module):
    def __init__(self,
                 in_features: int,
                 out_features: int,
                 activation="linear",
                 temperature=1.0):
        super().__init__()

        self.activation = activation
        self.temperature = temperature
        self.att = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)
        self.cla = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)

        self.bn_att = nn.BatchNorm1d(out_features)
        self.init_weights()

    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)
        init_bn(self.bn_att)

    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.clamp(self.att(x), -10, 10), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == 'linear':
            return x
        elif self.activation == 'sigmoid':
            return torch.sigmoid(x)


class AttBlockV2(nn.Module):
    def __init__(self,
                 in_features: int,
                 out_features: int,
                 activation="linear"):
        super().__init__()

        self.activation = activation
        self.att = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)
        self.cla = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)

        self.init_weights()

    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)

    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.tanh(self.att(x)), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == 'linear':
            return x
        elif self.activation == 'sigmoid':
            return torch.sigmoid(x)

In [8]:
class resnestSED(nn.Module):
    def __init__(
        self,
        base_model_name: str,
        pretrained=False,
        num_classes=24,
        spectrogram_params={},
        logmel_extractor_params={},
        spec_augmenter_params={},
        pce_params={}
    ):
        super().__init__()
        self.spectrogram_extractor = Spectrogram(**spectrogram_params)

        # Logmel feature extractor
        self.logmel_extractor = LogmelFilterBank(**logmel_extractor_params)
        
        #Pcen converter
        self.pcen_converter = spec.pcen(**pce_params)

        # Spec augmenter
        self.spec_augmenter = SpecAugmentation(**spec_augmenter_params)
        
        self.interpolate_ratio = 30  # Downsampled ratio
        self.mixup_alpha = 0.2
        self.random_state = np.random.RandomState(123)
        
        if pretrained:
            self.base_model = EfficientNet.from_pretrained(base_model_name)
        else:
            self.base_model = EfficientNet.from_name(base_model_name)

        in_features = self.base_model._fc.in_features

        self.fc1 = nn.Sequential(
            nn.Linear(1536, 1024), nn.ReLU(), nn.Dropout(p=0.5),
            nn.Linear(1024, num_classes, bias=True)
        )
        #self.att_block = AttBlockV2(in_features, num_classes, activation="sigmoid")

        self.init_weight()
        
    def mixup(self, x):
        sizws = x.size()
        #lam = torch.from_numpy(self.random_state.beta(self.mixup_alpha, self.mixup_alpha, (sizws[0], 1))).cuda()
        lam = self.random_state.beta(self.mixup_alpha, self.mixup_alpha, 1)[0]
        index = list(range(x.size(0)))
        random.shuffle(index)
        #out = (x.view(sizws[0], -1) * lam + x[index].squeeze().view(sizws[0], -1) * (1-lam)).view(sizws[0], sizws[1], sizws[2], sizws[3],)
        out = (x * lam + x[index].squeeze() * (1-lam))
        return out.float(), {'lam': lam, 'index': index}

    def init_weight(self):
        [init_layer(a) for a in self.fc1.children() if type(a) == torch.nn.modules.linear.Linear]


    def forward(self, input):        
        x = self.spectrogram_extractor(input)
        x = self.logmel_extractor(x)
        
        x_mels = self.logmel_extractor.power_to_db(x)
        x_pcen = self.pcen_converter(x) 
        x_clear = self.logmel_extractor.power_to_db(x ** 1.5)
        
        
        x = torch.cat((x_mels,x_pcen,x_clear),1)
        #x = torch.cat((x,x,x),1)
        
        frames_num = x.size(2)
        
        if self.training:
            x, mix_info = self.mixup(x)
            #x = self.spec_augmenter(x)
        else:
            mix_info = None
                
        # (batch_size, channels, freq, frames)
        x = self.base_model.extract_features(x)
        # (batch_size, channels, frames)
        x = torch.mean(x, dim=3)

        # channel smoothing
        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        x = x1 + x2
        
        x = torch.mean(x, dim=2)
        x = self.fc1(x)
        
        return x, torch.sigmoid(x), mix_info
        
        """
        # (batch_size, channels, frames)
        x = torch.mean(x, dim=3)

        # channel smoothing
        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        x = x1 + x2

        x = F.dropout(x, p=0.5, training=self.training)
        x = x.transpose(1, 2)
        x = F.relu_(self.fc1(x))
        x = x.transpose(1, 2)
        x = F.dropout(x, p=0.5, training=self.training)
        (clipwise_output, norm_att, segmentwise_output) = self.att_block(x)
        logit = torch.sum(norm_att * self.att_block.cla(x), dim=2)
        segmentwise_logit = self.att_block.cla(x).transpose(1, 2)
        segmentwise_output = segmentwise_output.transpose(1, 2)

        # Get framewise output
        framewise_output = interpolate(segmentwise_output,
                                       self.interpolate_ratio)
        framewise_output = pad_framewise_output(framewise_output, frames_num)

        framewise_logit = interpolate(segmentwise_logit, self.interpolate_ratio)
        framewise_logit = pad_framewise_output(framewise_logit, frames_num)

        output_dict = {
            "framewise_output": framewise_output,
            "segmentwise_output": segmentwise_output,
            "logit": logit,
            "framewise_logit": framewise_logit,
            "clipwise_output": clipwise_output
        }

        return output_dict, mix_info
        """

In [9]:
# LRAP. Instance-level average
# Assume float preds [BxC], labels [BxC] of 0 or 1
def LRAP(preds, labels):
    # Ranks of the predictions
    ranked_classes = torch.argsort(preds, dim=-1, descending=True)
    # i, j corresponds to rank of prediction in row i
    class_ranks = torch.zeros_like(ranked_classes)
    for i in range(ranked_classes.size(0)):
        for j in range(ranked_classes.size(1)):
            class_ranks[i, ranked_classes[i][j]] = j + 1
    # Mask out to only use the ranks of relevant GT labels
    ground_truth_ranks = class_ranks * labels + (1e6) * (1 - labels)
    # All the GT ranks are in front now
    sorted_ground_truth_ranks, _ = torch.sort(ground_truth_ranks, dim=-1, descending=False)
    pos_matrix = torch.tensor(np.array([i+1 for i in range(labels.size(-1))])).unsqueeze(0)
    score_matrix = pos_matrix / sorted_ground_truth_ranks
    score_mask_matrix, _ = torch.sort(labels, dim=-1, descending=True)
    scores = score_matrix * score_mask_matrix
    score = (scores.sum(-1) / labels.sum(-1)).mean()
    return score.item()

# label-level average
# Assume float preds [BxC], labels [BxC] of 0 or 1
def LWLRAP(preds, labels):
    # Ranks of the predictions
    ranked_classes = torch.argsort(preds, dim=-1, descending=True)
    # i, j corresponds to rank of prediction in row i
    class_ranks = torch.zeros_like(ranked_classes)
    for i in range(ranked_classes.size(0)):
        for j in range(ranked_classes.size(1)):
            class_ranks[i, ranked_classes[i][j]] = j + 1
    # Mask out to only use the ranks of relevant GT labels
    ground_truth_ranks = class_ranks * labels + (1e6) * (1 - labels)
    # All the GT ranks are in front now
    sorted_ground_truth_ranks, _ = torch.sort(ground_truth_ranks, dim=-1, descending=False)
    # Number of GT labels per instance
    num_labels = labels.sum(-1)
    pos_matrix = torch.tensor(np.array([i+1 for i in range(labels.size(-1))])).unsqueeze(0)
    score_matrix = pos_matrix / sorted_ground_truth_ranks
    score_mask_matrix, _ = torch.sort(labels, dim=-1, descending=True)
    scores = score_matrix * score_mask_matrix
    score = scores.sum() / labels.sum()
    return score.item()

def mixup_loss(cor, x, y, mix_info):
    clip_loss = cor(x['clipwise_output'], y) * mix_info['lam'] + cor(x['clipwise_output'], y[mix_info['index']].squeeze()) * (1-mix_info['lam'])
    frame_loss = cor(x['framewise_output'].max(dim=1)[0], y) * mix_info['lam'] + cor(x['framewise_output'].max(dim=1)[0], y[mix_info['index']].squeeze()) * (1-mix_info['lam'])
    return 0.5*clip_loss+frame_loss

def mixup_socre(cor, x, y, mix_info):
    return cor(x, y) * mix_info['lam'] + cor(x, y[mix_info['index']].squeeze()) * (1-mix_info['lam'])

In [10]:
train_tp = pd.read_csv('../../data/train_tp.csv')
train_fp = pd.read_csv('../../data/train_fp.csv')
submission = pd.read_csv('../../data/sample_submission.csv')

pred_target = list(submission.columns)[1:]
SR = settings['SR']

skf = StratifiedKFold(n_splits=settings['n_splits'], shuffle=True, random_state=42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
tta = np.zeros((len(submission), 24))
cv_score = 0

for fold_id, (train_index, val_index) in enumerate(skf.split(train_tp, train_tp.species_id)):
    print(f'---------- fold {fold_id} ----------')
    
    model = resnestSED(**settings['model_params']).to(device)
    optim = Adam(model.parameters(), **settings['optim_params'])
    #scheduler = lr_scheduler.ReduceLROnPlateau(optimizer=optim, **scheduler_params)
    #scheduler = lr_scheduler.CosineAnnealingLR(optimizer=optim, T_max=epochs-10)
    scheduler = wu.SimpleCosineAnnealingWarmup(optim, **settings['scheduler_params'])
    
    pos_weights = torch.ones(24)
    pos_weights = pos_weights * 24
    criterion = nn.BCELoss()#pos_weight=pos_weights)
    #criterion = FocalLoss()
    #criterion = cr.ImprovedFocalLoss(settings['loss_params'])
    
    train_dataset = RfcxDataSet(train_tp.iloc[train_index], **settings['train_params'])
    val_dataset   = RfcxDataSet(train_tp.iloc[val_index], **settings['val_params'])

    train_dataloader = DataLoader(train_dataset, shuffle=True, **settings['dataloder_params'])
    val_dataloader = DataLoader(val_dataset, shuffle=False, **settings['dataloder_params'])
    
    es = 50
    bast_score = 0
    for epoch in range(1, settings['epochs']):
        if es <= 0:
            break
        
        start_time = time.time()
        
        # train
        model.train()
        train_loss = 0
        train_score = 0
        train_corr = 0
        
        for data in train_dataloader:
            image = data[0].float().to(device)
            label = data[1]
            
            optim.zero_grad()
            x, x_sig, mix_info = model(image)
            x, x_sig = x.cpu(), x_sig.cpu()
            
            #output = {k:v.cpu() for k,v in output.items()}
            #pred_labels = output["framewise_output"].max(1)[0]
            
            loss = mixup_socre(criterion, x_sig, label, mix_info)
            #loss = mixup_socre(criterion, output['clipwise_output'], label, mix_info)
            #loss = criterion(output[''], label, mix_info)
            score = mixup_socre(LWLRAP, x_sig, label, mix_info)
            
            #score = LWLRAP(pred_labels, label)
                        
            loss.backward()
            optim.step()
            
            vals, answers = torch.max(x_sig, 1)
            vals, targets = torch.max(label, 1)
            vals, targets2 = torch.max(label[mix_info['index']], 1)
            
            corrects = 0
            for i in range(0, len(answers)):
                if answers[i] == targets[i]:
                    corrects = corrects + 1
                if answers[i] == targets2[i]:
                    corrects = corrects + 1
                
                corrects = 1 if corrects > 0 else 0
                    
            
            train_corr += corrects
            train_loss += loss.item()
            train_score += score
            
        train_loss  /= len(train_dataloader)
        train_score /= len(train_dataloader)
        
        # val
        model.eval()
        val_loss = 0
        val_score = 0
        val_corr = 0

        with torch.no_grad():
            for val_data in val_dataloader:
                image = val_data[0].float().to(device)
                label = val_data[1]

                x, x_sig, mix_info = model(image)
                x, x_sig = x.cpu(), x_sig.cpu()
                #output = {k:v.cpu() for k,v in output.items()}
                
                #pred_labels = output["framewise_output"].max(1)[0]
                vals, answers = torch.max(x_sig, 1)
                vals, targets = torch.max(label, 1)
                
                corrects = 0
                for i in range(0, len(answers)):
                    if answers[i] == targets[i]:
                        corrects = corrects + 1

                val_corr += corrects
                val_loss += criterion(x_sig, label)
                val_score += LWLRAP(x_sig, label)
                
        val_loss  /= len(val_dataloader)
        val_score /= len(val_dataloader)
        
        duration = str(datetime.timedelta(seconds=time.time() - start_time))[:7]
        print(f'E {epoch:3}| T | L: {train_loss:.3} | S: {train_score:.3} | C: {train_corr}/{len(train_dataset)} | V | L: {val_loss:.3} | S: {val_score:.3} | C: {val_corr}/{len(val_dataset)} | T: {duration} | es: {es}')

        if bast_score < val_score:
            bast_score = val_score
            bast_path = save(fold_id, model, optim, criterion)
        else:
            es -= 1
        
        if es <= 0:
            break
            
        scheduler.step(val_score)
    
    print(f"bast score: {bast_score}")
    cv_score += bast_score / settings['n_splits']
    del model, train_dataset, val_dataset, train_dataloader, val_dataloader, optim
    gc.collect()

---------- fold 0 ----------
Loaded pretrained weights for efficientnet-b3
E   1| T | L: 0.247 | S: 0.187 | C: 51/972 | V | L: 0.309 | S: 0.163 | C: 8/244 | T: 0:00:53 | es: 50
E   2| T | L: 0.169 | S: 0.325 | C: 60/972 | V | L: 0.161 | S: 0.414 | C: 64/244 | T: 0:00:51 | es: 50
E   3| T | L: 0.137 | S: 0.524 | C: 61/972 | V | L: 0.128 | S: 0.619 | C: 111/244 | T: 0:00:52 | es: 50
E   4| T | L: 0.113 | S: 0.65 | C: 61/972 | V | L: 0.145 | S: 0.614 | C: 116/244 | T: 0:00:51 | es: 50
E   5| T | L: 0.108 | S: 0.672 | C: 61/972 | V | L: 0.1 | S: 0.723 | C: 144/244 | T: 0:00:51 | es: 49
E   6| T | L: 0.097 | S: 0.721 | C: 61/972 | V | L: 0.0873 | S: 0.776 | C: 161/244 | T: 0:00:52 | es: 49
E   7| T | L: 0.0848 | S: 0.765 | C: 61/972 | V | L: 0.0763 | S: 0.773 | C: 158/244 | T: 0:00:51 | es: 49
E   8| T | L: 0.08 | S: 0.793 | C: 61/972 | V | L: 0.0727 | S: 0.803 | C: 165/244 | T: 0:00:51 | es: 48
E   9| T | L: 0.0786 | S: 0.788 | C: 61/972 | V | L: 0.0645 | S: 0.833 | C: 176/244 | T: 0:00:52

In [12]:
print(cv_score)

0.8820598095655441


In [26]:
def prediction_for_clip(audio_id: str,
                        clip: np.ndarray, 
                        model: resnestSED,
                        threshold=0.5):
    PERIOD = settings['duration']
    audios = []
    y = clip.astype(np.float32)
    len_y = len(y)
    stride = 2
    start = 0
    end = PERIOD * SR
    while True:
        y_batch = y[start:end].astype(np.float32)
        
        start += stride * SR #int(end - ((settings['duration']/2) * SR))
        end += stride * SR #int((settings['duration']/2) * SR)
        #mel = create_mel(y_batch)
        audios.append(y_batch)
        
        if len_y < end:
            break
            
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    array = np.asarray(audios)
    image = torch.from_numpy(array).to(device)
    
    model.eval()
    estimated_event_list = []
    global_time = 0.0
    

    with torch.no_grad():
        x, x_sig, _ = model(image)
        
    return x_sig.max(0)[0].cpu().numpy()

In [27]:
def prediction(test_df: pd.DataFrame,
               model: dict,
               threshold=0.5):
    #model = get_model(model_config, weights_path)
    unique_audio_id = test_df.recording_id.unique()

    warnings.filterwarnings("ignore")
    prediction_dfs = []
    frame_dict = dict()
    
    for audio_id in tqdm(unique_audio_id):
        clip, orig_sr = sf.read(f'/home/yuigahama/kaggle/rfcx/data/test/{audio_id}.flac')
        
        frame_pred = prediction_for_clip(
            audio_id,
            clip=clip,
            model=model,
            threshold=threshold
        )
        frame_dict[audio_id] = frame_pred
    return frame_dict

In [28]:
preds = []
frams = [] 
for i in range(5):

    model = resnestSED(**settings['model_params']).to(device)
    params = torch.load(f'/home/yuigahama/kaggle/rfcx/model/{settings["test_name"]}/{settings["test_name"]}_{i}.model')
    model.load_state_dict(params['model_state_dict'])
    
    frame_dict = prediction(
        test_df=submission,
        model=model,
        threshold=0.5
    )
    frams.append(pd.DataFrame(frame_dict).T)

  0%|          | 0/1992 [00:00<?, ?it/s]

Loaded pretrained weights for efficientnet-b3


  0%|          | 0/1992 [00:00<?, ?it/s]

Loaded pretrained weights for efficientnet-b3


100%|██████████| 1992/1992 [08:34<00:00,  3.87it/s]


...

  0%|          | 0/1992 [00:00<?, ?it/s]

Loaded pretrained weights for efficientnet-b3


  0%|          | 0/1992 [00:00<?, ?it/s]

Loaded pretrained weights for efficientnet-b3


100%|██████████| 1992/1992 [08:40<00:00,  3.83it/s]


In [29]:
sub = pd.DataFrame(np.zeros((len(submission), 24)),columns=pred_target, index=submission['recording_id'])
for p,j in zip(frams, [0.2, 0.2, 0.2, 0.2, 0.2]):
    p.columns = pred_target
    sub += (p * j)
#sub /= 5
sub.reset_index().to_csv(f'/home/yuigahama/kaggle/rfcx/model/{settings["test_name"]}/submission_{settings["test_name"]}.csv', index=False)
sub

,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,...,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23
recording_id,,,,,,,,,,,,,,,,,,,,,
000316da7,0.031325,0.005718,0.004262,0.881732,0.002063,0.019854,0.004019,0.000568,0.005294,0.006470,...,0.027642,0.020944,0.013321,0.004221,0.017528,0.002623,0.004209,0.001297,0.007740,0.012898
003bc2cb2,0.001193,0.015094,0.024449,0.687894,0.005903,0.000990,0.001214,0.003257,0.001508,0.000755,...,0.013368,0.002896,0.994010,0.002935,0.001803,0.001644,0.001551,0.001312,0.002568,0.002471
0061c037e,0.006597,0.007180,0.010966,0.592661,0.000918,0.008441,0.005724,0.197479,0.000644,0.002271,...,0.001086,0.004660,0.018047,0.028849,0.000831,0.003876,0.065746,0.006939,0.005750,0.024470
010eb14d3,0.985422,0.001623,0.002059,0.002416,0.002548,0.002703,0.001334,0.001193,0.849815,0.001026,...,0.001783,0.002650,0.002213,0.000764,0.118122,0.000757,0.000659,0.002717,0.001723,0.001526
011318064,0.003240,0.001418,0.006081,0.293382,0.003450,0.023126,0.004695,0.001466,0.002550,0.001682,...,0.941865,0.636251,0.007206,0.007521,0.098871,0.001977,0.003397,0.002408,0.002297,0.002381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ff68f3ac3,0.082995,0.002622,0.005282,0.206957,0.025098,0.785751,0.003826,0.018418,0.003600,0.002164,...,0.002132,0.951679,0.005041,0.003695,0.012138,0.008987,0.014031,0.002809,0.003063,0.979039
ff973e852,0.003542,0.002430,0.003153,0.094800,0.002200,0.009852,0.032946,0.861362,0.002960,0.171471,...,0.005930,0.846950,0.003446,0.162376,0.003956,0.006305,0.040521,0.003243,0.226056,0.015171
ffa5cf6d6,0.008055,0.010493,0.020753,0.818032,0.000935,0.005503,0.005158,0.015394,0.000568,0.064747,...,0.002757,0.989497,0.003788,0.055946,0.000724,0.003924,0.011274,0.000701,0.004423,0.017492


In [ ]:
settings["test_name"]

In [ ]:
len(np.arange(7e-5,8.0e-4,8e-5).tolist())

In [ ]:
len(np.arange(1e-5,1.0e-3,7e-5).tolist())

In [ ]:
print(torch.tensor([[1],[2],[3]]).size())
a*torch.tensor([[1],[2],[3]])

In [ ]:
np.arange(0.0001,0.0008,0.00015)